# Questions

- What does it mean to "model" an antibody? Do we just give an amino acid sequence? Should we return some kind of genomic sequence?
    - For example, what if we wanted to return a family of antibodies with some common characteristics?
    - Should we return **how** to make the antibody? Like some sort of manufacturing process to actually create the thing we think works?
- How does an antibody work?
- What publically available data do we have for naturally occurring and synthetic antibodies?
- What biologically relevant, predictive properties do these data sets have?

# Assumptions

For now let's simplify the model to predicting the amino acid sequence of the antigen binding site (paratope) within the antibody given information about the antigen and priors about paratope possibilities. The information about the antigen includes information about it's primary, secondary, and tertiary structure. We will avoid antigens with quaternary structure (i.e. coordinating subunits).

# Modeling the Problem

The most straightforward way I can think to approach this problem is to reduce the sample space of possible antibodies based on information about the antigen and then use a molecular dynamics simulations to determine the best possible antigen after that. I believe many of the machine learning applications will be in reducing the sample space of antibodies rather than replacing the simulation aspect. 

$$\text{All permutations of amino acids in paratope}$$
$$\downarrow$$
$$\text{Possible paratopes from V(D)J Recombination}$$
$$\downarrow$$
$$\text{Most likley paratopes based on Markov model with some cutoff }\alpha$$
$$\downarrow$$
$$\text{Antigen specific paratopes}$$
$$\downarrow$$
$$\text{Molecular dynamics simulations}$$

We'll use the SARS-CoV-2 spike glycoprotein as an example antigen. It's obviously still very relevant and quite well studied plus we can compare our results to available antibody treatments to see how well our models perform. Namely we can look at how "far" our predictions are from the following antibodies known to bind to the SARS-CoV-2 spike glycoprotein
- https://www.rcsb.org/structure/7bwj
- https://www.rcsb.org/structure/7bz5
- https://www.rcsb.org/structure/7CR5
- https://www.rcsb.org/structure/7cm4

I'm hesitant to take a purely machine learning approach to this problem because I'm not sure what an appropriate loss function would be in this case. We can't use antigen-antigen distance metrics nor can we use antibody-antibody distance metrics because neither of those capture the antibody-antigen complex relationsip. Furthermore, accurate bonding between antibodies and antigens isn't necessarily predictive across different antibodies and antigens. We would need to map that binding to some sort of high-dimensional, potentially unintelligible by humans, latent space within which we could find global and local extrema. 

# Implementation

## Completely Naive Paratope Sample Space

Each paratope consists of approximately [15 amino acids](https://www.ncbi.nlm.nih.gov/books/NBK2396/#:~:text=Each%20paratope%20has%20about%2015,than%20particular%20amino%20acid%20compositions.). Under a completely naive assumption, each amino acid can appear independent of any other amino acid. As such, we can use the counting principle to derive an upper bound:

$$(20)^{15} = 3.2768e19 $$

Ok so this is a lot of possibilities and some of these are obviously irrelevant. For example, imagine a sequence entirely composed of glycines. This would probably not exist as a possible antibody state. Let's further filter by considering V(D)J recombination.

## V(D)J Recombination

In V(D)J recombination, developing lymphocytes combine different variable (V), diversity (D), and joining (J) regions to produce a wide array of antibodies. Heavy chains incorporate diversity regions whereas light chains do not. Light chains can recombine gene segments from $\kappa$ regions or $\lambda$ regions. There are [200 possible $\kappa$ light chains and 120 possible $\lambda$ light chains](https://www.ncbi.nlm.nih.gov/books/NBK27140/#:~:text=Many%20different%20V%20regions%20can,200%20different%20V%CE%BA%20regions.). However, since there are ~11,000 possible heavy chains there are ~3.52e6 possible antibodies that can be generated in humans. Since we're focusing on light chain antibodies, we'll only look at the 320 possibilities for light chains for now. Within that we can make a further simplification since we're only concerned with the antigen binding site (aka paratope). That is, we can focus on the different variable regions exclusively since those are what actually bind to our antigen. Now we only have to consider [70 variable regions](https://www.ncbi.nlm.nih.gov/books/NBK27140/) (40 $V_{\kappa}$ + 30 $V_{\lambda}$).

### What information are we losing?

Narrowing our search space from $3.2768e19 \rightarrow 70$ should make us question what kind of information we're losing out on. What's the consequence of ignoring joining regions in light chains? Of ignoring heavy chains all together? In short there is a ton of information we're losing out on. Namely the heavy chain exists to help antibodies become embedded in transmembrane regions and there are even cases where heavy-chain antibodies can themselves [bind to certain antigens](https://en.wikipedia.org/wiki/Heavy-chain_antibody#:~:text=A%20heavy%2Dchain%20antibody%20is,VH%20and%20VL)! This information would be incredibly important for predicting things like T-cell or B-cell function. Perhaps designing heavy-chain exclusive antibodies is much cheaper or more effective. In our case, we're only focused on binding and ignoring all other factors. 

## Markov Modeling of Light Chain Sequences

Let's use existing amino acid sequences from light chains to determine the transition probabilities for amino acids. We can then use random walks to create additional candidates for simulation.

In [1]:
import numpy as np

In [ ]:
# load amino acid sequences of variable regions

In [ ]:
def calculateTransitionProbs(_kappa, _lambda)
    light_chain_variable = _lambda + _kappa
    counts = {}

    for v in light_chain_variable:
        for i in range(len(v) - 1):
            if v[i:i+1] not in counts:
                counts[v[i:i+1]] = 1
            else:
                counts[v[i:i+1]] += 1
        
    normalizing_factors = {}
    
    for pair, count in counts.items():
        if pair[0] not in counts:
            counts[pair[0]] = 1
        else:
            counts[pair[0]] += 1
            
    for c in counts:
        